In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Data/final_metadata.csv')

In [3]:
df.shape

(19951, 14)

In [4]:
df.drop(columns=['release_date','vote_average', 'vote_count'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19951 entries, 0 to 19950
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 19951 non-null  int64  
 1   title              19951 non-null  object 
 2   genres             19951 non-null  object 
 3   original_language  19951 non-null  object 
 4   overview           19951 non-null  object 
 5   popularity         19951 non-null  float64
 6   keywords           19951 non-null  object 
 7   year               19951 non-null  int64  
 8   cast               19951 non-null  object 
 9   director           19951 non-null  object 
 10  score              19951 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.7+ MB


In [6]:
#Combine title, synopsis, and Genre
df['soup'] = df.apply(lambda row: f"Title: {row['title']}. Genres: {row['genres']}. Keywords: {row['keywords']}. Cast: {row['cast']}. Director: {row['director']}.", axis=1)
df['soup'][0]

'Title: Godzilla x Kong: The New Empire. Genres: Science Fiction Action Adventure. Keywords: giantmonster sequel dinosaur kaiju fantasyworld giantape godzilla kingkong mongkey. Cast: RebeccaHall BrianTyreeHenry DanStevens. Director: Adam Wingard.'

In [7]:
# pip install -U langchain
# pip install -U langchain-community

In [8]:
from langchain.docstore.document import Document

movies = []

for index, row in df.iterrows():
    x = Document(page_content=row['soup'], 
                 metadata={
                     "movie": row['title'],
                     "language": row['original_language'], 
                     "popularity": row['popularity'], 
                     "year": row['year'],
                     "synopsis": row['overview'],
                     "score": row['score']
                     })
    movies.append(x)

# doc =  Document(page_content="text", metadata={"source": "local"})

In [9]:
len(movies)

19951

In [10]:
import weaviate
from langchain.vectorstores import Weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [11]:
import weaviate
from weaviate.auth import AuthApiKey

client = weaviate.connect_to_weaviate_cloud(
    cluster_url="",                       # `weaviate_url`: your Weaviate URL
    auth_credentials=AuthApiKey(""),      # `weaviate_key`: your Weaviate API key
)

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name, 
  model_kwargs=model_kwargs
)

c:\Users\aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
df.columns

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'keywords', 'year', 'cast', 'director', 'score', 'soup'],
      dtype='object')

In [14]:
vector_db = WeaviateVectorStore(
    client = client,
    index_name = "LangChain_2adbdf1cd46e49c0ae3561cd1aa07cbe",
    text_key = "text",
    embedding = embeddings
)

In [15]:
soups = pd.Series(df['soup'].values, index=df['title'])

In [16]:
soups.head()

title
Godzilla x Kong: The New Empire     Title: Godzilla x Kong: The New Empire. Genres...
Meg 2: The Trench                   Title: Meg 2: The Trench. Genres: Action Scien...
The Pope's Exorcist                 Title: The Pope's Exorcist. Genres: Horror Mys...
Transformers: Rise of the Beasts    Title: Transformers: Rise of the Beasts. Genre...
Dune: Part Two                      Title: Dune: Part Two. Genres: Science Fiction...
dtype: object

In [21]:
def get_recommendations(title):
    
    if title not in soups:
        raise ValueError(f"Title '{title}' not found in indices")
    
    query = soups.get(title)  
    
    results = vector_db.similarity_search(query, k=11)

    top_ten = []

    for x in results[1:]:
        movie_metadata = {
            'movie': x.metadata['movie'],
            'language': x.metadata['language'],
            'popularity': x.metadata['popularity'],
            'score': round(x.metadata['score'],1),
            'synopsis': x.metadata['synopsis'],
            'year': x.metadata['year']
        }
        top_ten.append(movie_metadata)

    df_top_ten = pd.DataFrame(top_ten)
    df_top_ten = df_top_ten.sort_values(by=['score', 'popularity'], ascending=[False, False])[['movie', 'language','score','year']]

    return df_top_ten

  

In [23]:
get_recommendations('The Dark Knight Rises')

,movie,language,score,year
0,The Dark Knight,English,8.5,2008.0
2,Batman Begins,English,7.7,2005.0
5,The Batman,English,7.6,2022.0
4,"Batman: The Dark Knight Returns, Part 2",English,7.5,2013.0
3,"Batman: The Dark Knight Returns, Part 1",English,7.3,2012.0
8,Batman,English,7.2,1989.0
1,Batman: The Dark Knight Returns,English,6.7,2013.0
6,Batman: Gotham Knight,English,6.6,2008.0
9,Batman: Gotham by Gaslight,English,6.6,2018.0
7,Knights of Badassdom,English,6.2,2013.0


In [24]:
get_recommendations('Hulk')

,movie,language,score,year
2,Planet Hulk,English,6.7,2010.0
8,Hulk vs. Wolverine,English,6.6,2009.0
6,Hulk vs. Thor,English,6.6,2009.0
4,The Trial of the Incredible Hulk,English,6.5,1989.0
7,Hulk Vs,English,6.5,2009.0
3,Hulk: Where Monsters Dwell,English,6.5,2016.0
1,The Incredible Hulk,English,6.4,1977.0
9,The Incredible Hulk Returns,English,6.4,1988.0
5,The Death of the Incredible Hulk,English,6.3,1990.0
0,The Incredible Hulk,English,6.2,2008.0


In [25]:
get_recommendations('Your Name')

,movie,language,score,year
6,Weathering with You,Japanese,7.6,2019.0
8,Paprika,Japanese,7.5,2006.0
2,Given,Japanese,7.1,2020.0
7,Summer Wars,Japanese,7.1,2009.0
5,Drifting Home,Japanese,6.8,2022.0
3,Hello World,Japanese,6.7,2019.0
0,Her Blue Sky,Japanese,6.6,2019.0
9,Orange: Future,Japanese,6.6,2016.0
4,Urusei Yatsura: Beautiful Dreamer,Japanese,6.5,1984.0
1,Hal,Japanese,6.5,2013.0


In [26]:
get_recommendations('The Godfather')

,movie,language,score,year
0,The Godfather Part II,English,8.5,1974.0
2,The Godfather Part III,English,7.3,1990.0
6,The Traitor,Italian,7.3,2019.0
1,The Godfather Trilogy: 1901-1980,English,6.9,1992.0
8,The Sicilian Clan,French,6.8,1969.0
9,The Italian Connection,Italian,6.6,1972.0
5,Salvatore Giuliano,Italian,6.6,1962.0
3,Our Godfather,English,6.5,2019.0
4,The Sicilian,English,6.2,1987.0
7,Gotti,English,5.9,2018.0


In [27]:
get_recommendations('Suzume')

,movie,language,score,year
1,Your Name,Japanese,8.4,2016.0
7,Akira,Japanese,7.7,1988.0
0,Jujutsu Kaisen 0,Japanese,7.5,2021.0
9,Digimon Adventure: Last Evolution Kizuna,Japanese,6.9,2020.0
5,House,Japanese,6.9,1977.0
2,Saint Seiya Heaven Chapter: Overture,Japanese,6.8,2004.0
6,Kizumonogatari Part 1: Tekketsu,Japanese,6.8,2016.0
3,Jiang Ziya: Legend of Deification,Chinese,6.7,2020.0
8,Roujin Z,Japanese,6.5,1991.0
4,Harmony,Japanese,6.5,2015.0


In [28]:
get_recommendations('Inception')

,movie,language,score,year
9,Interstellar,English,8.4,2014.0
7,Donnie Darko,English,7.7,2001.0
6,Fantastic Planet,French,7.2,1973.0
5,Interstellar: Nolan's Odyssey,English,6.7,2014.0
4,Trance,English,6.6,2013.0
1,In My Dreams,English,6.5,2015.0
0,Gandahar,French,6.5,1987.0
2,Brainstorm,English,6.4,1983.0
3,Infini,English,6.1,2015.0
8,Hudson Hawk,English,6.0,1991.0


In [29]:
get_recommendations('Interstellar')

,movie,language,score,year
9,2001: A Space Odyssey,English,8.0,1968.0
1,The Martian,English,7.7,2015.0
5,Arrival,English,7.5,2016.0
8,Passengers,English,6.9,2016.0
0,Interstellar: Nolan's Odyssey,English,6.7,2014.0
4,Star Trek: The Motion Picture,English,6.5,1979.0
6,Millennium,English,6.3,1989.0
3,Capsule,English,6.3,2015.0
7,Approaching the Unknown,English,6.1,2016.0
2,Lost in Space,English,5.8,1998.0
